<p><strong><span style="font-size: 20px; color: rgb(124, 112, 107);">Background</span></strong></p>
<p><span style="color: rgb(0, 0, 0);">The data obtained from Kaggle. Propertyid, location_id, page_url property_type, price, location, city, province_name, latitude, longitude, baths, area, pupose, bedrooms, date_added,agency, and agent are all included in the dataset. This data spans the periods 2018-8 to 2019-8.</span></p>

In [ ]:
import math
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams["figure.figsize"] = (20,10)

import missingno as msno

# !pip install xlrd

# !pip install autoviz
from autoviz.AutoViz_Class import AutoViz_Class

# !pip install dtale
import dtale

import folium
from folium.plugins import MarkerCluster

from scipy import stats

from sklearn.tree import DecisionTreeRegressor

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#Check the shape of the dataset after loading it.
df = pd.read_csv('/kaggle/input/zameencom-property-data-pakistan/Property_with_Feature_Engineering.csv')
df.shape

In [ ]:
df.sample(5)

### EDA

In [ ]:
df1 = df.drop(['property_id', 'location_id', 'page_url', 'area', 'area_marla', 'locality', 'agency', 'agent'], axis='columns')
df1.shape

In [ ]:
#Remove the day and save it in the date_added column.
df1['date_added'] = pd.to_datetime(df1['date_added'])
df1 = df1.reset_index()
df1['date_added'] = df1['date_added'].apply(lambda x: x.strftime('%Y-%m'))

In [ ]:
#This function removes outliers from one or more columns.
def remove_outlier(df, column_lst):
    df_in = df
    for col in column_lst:
        q1 = df_in[col].quantile(0.25)
        q3 = df_in[col].quantile(0.75)
        iqr = q3-q1
        fence_low  = q1-1.5*iqr
        fence_high = q3+1.5*iqr
        df_in = df_in.loc[(df_in[col] > fence_low) & (df_in[col] < fence_high)]
    return df_in

In [ ]:
#Boxplots from one or more columns are shown using this function.
def show_graphs(df, lst, title=""):
    if len(lst) > 1:
        fig, axs = plt.subplots(1, len(lst), figsize=(16, 5))
        for i in range(0, len(lst)):
            axs[i].boxplot(df[lst[i]])
            axs[i].set_title(lst[i])
        fig.suptitle(title)
    else:
        fig, axs = plt.subplots(figsize=(16, 8))
        axs.boxplot(df[lst[0]])
        axs.set_title(lst[0])
        fig.suptitle(title)

In [ ]:
#Examine the values that are missing.
msno.matrix(df1)

In [ ]:
df1.sample(5)

In [ ]:
#Check the Outliers
show_graphs(df1,['price','baths','area_sqft'])

In [ ]:
#Remove Ouliers
df2 = remove_outlier(df1, ['price','baths','area_sqft'])

In [ ]:
#Check the Outliers
show_graphs(df2,['price','baths','area_sqft'])

In [ ]:
#Add a column called price_per_sqft that contains price divided area_sqft values.
df3 = df2.copy()
df3['price_per_sqft'] = df2['price']/df2['area_sqft']

In [ ]:
#Check the Outliers
show_graphs(df3,['price', 'area_sqft', 'price_per_sqft'])

In [ ]:
#Remove Outliers
df4 = remove_outlier(df3, ['price_per_sqft'])

In [ ]:
#Check the Outliers
show_graphs(df4,['price', 'area_sqft', 'price_per_sqft'])

In [ ]:
df4.to_csv("df4.csv",index=False)

In [ ]:
#EDA using Autoviz
autoviz = AutoViz_Class().AutoViz('df4.csv')

In [ ]:
#EDA using dtale Lib
dtale.show(df4)

In [ ]:
# Remove outliers
index = df4[(df4['baths']==0) & (df4['bedrooms']>1)].index
df5 = df4.drop(index) 

index = df5[(df5['property_type']=='Room') & ((df5['baths']>1) | (df5['bedrooms']>1))].index
df6 = df5.drop(index)

index = df6[(df6['bedrooms']==0)].index
df7 = df6.drop(index)

index = df7[((df7['bedrooms'])+3)<df7['baths']].index
df8 = df7.drop(index)

In [ ]:
#Remove outliers from Square Feet, Bedrooms and bathrooms
def remove_outliers(df,p_type,area,bedrooms,baths):
    sqft = np.sort(df[area].unique())
    for sq in sqft:
        rooms = int(((0.5*sq)/171)+2)
        df.loc[(df[p_type]!='Room') & (df[area]==sq) & (df[bedrooms]>rooms), [bedrooms]] = rooms
        df.loc[(df[p_type]!='Room') & (df[area]==sq) & (df[baths]>(rooms+2)), [baths]] = rooms+2

remove_outliers(df8,'property_type','area_sqft','bedrooms','baths')

In [ ]:
#The available data includes both selling and renting items, but I'm only interested in selling items because buyers are aware of the true value of homes, flats, and other properties based on location and rooms.
#That's why i drop the Rental Rows and save into df9 
index = df8[df8['purpose']=='For Rent'].index
df9 = df8.drop(index)

#Since there are so many outliers in the price_bin column, it should be removed.
df10 = df9.drop(['price_bin'], axis='columns')

#Add a price_bin column based on this criterion.
df10['price_bin'] = 'Low'
df10.loc[(df10['price']>2500000) & (df10['price']<=5000000),['price_bin']] = 'Medium'
df10.loc[(df10['price']>5000000) & (df10['price']<=10000000),['price_bin']] = 'High'
df10.loc[(df10['price']>10000000),['price_bin']] = 'Very High'

#Drop below 50 thousand price rows
df11 = df10[df10['price']>50000]

In [ ]:
#Check the Outliers
show_graphs(df11,['price', 'area_sqft', 'price_per_sqft'])

#These columns are now better at predicting unseen data by using machine learning algorithms.

### Visualization

##### By date, sell a house, a farm house, a flat, and other things.

In [ ]:
def selling_by_date(df):
    df = df.sort_values(by="date_added")
    date = df['date_added'].unique()
    p_type_dict = {}
    grouped_p_type = df.groupby('property_type')
    for pt in grouped_p_type.groups.keys():
        lst = []
        for dt in date:
            df_dt = df[(df["date_added"] == dt) & (df['property_type']==pt)]
            df_dt_sum = len(df_dt)

            if df_dt_sum > 0:
                lst.append(df_dt_sum)
            else:
                lst.append(0)
        p_type_dict[pt] = lst
    
    return date, p_type_dict

In [ ]:
labels, p_type = selling_by_date(df11)

x = np.arange(len(labels))
width = 0.35
fig, ax = plt.subplots(len(p_type), 1, figsize=(16, 60),)
count = 0 
for pt in p_type.keys():
    ax[count].plot(x - width/2, p_type.get(pt), label = pt)
    ax[count].set_xticks(x)
    ax[count].set_xticklabels(labels, rotation=90)
    ax[count].legend()
    count+=1

    
plt.show()

In [ ]:
def heatMap(df):
    #Create Correlation df
    corr = df.corr()
    #Plot figsize
    fig, ax = plt.subplots(figsize=(20, 10))
    #Generate Color Map
    colormap = sns.diverging_palette(220, 10, as_cmap=True)
    #Generate Heat Map, allow annotations and place floats in map
    sns.heatmap(corr, cmap=colormap, annot=True, fmt=".2f")
    #Apply xticks
    plt.xticks(range(len(corr.columns)), corr.columns);
    #Apply yticks
    plt.yticks(range(len(corr.columns)), corr.columns)
    #show plot
    plt.show()
    
heatMap(df11)

### Use one Hot Encoding

In [ ]:
dummy1 = pd.get_dummies(df11.property_type)
dummy2 = pd.get_dummies(df11.price_bin)
dummy3 = pd.get_dummies(df11.location)
dummy4 = pd.get_dummies(df11.city)
dummy5 = pd.get_dummies(df11.province_name)

df12 = pd.concat([df11.drop(['property_type','price_bin','location','latitude','longitude','city','province_name','purpose','date_added','year','month','day'], axis='columns'),dummy1,dummy2,dummy3,dummy4,dummy5],axis='columns')
df12.sample(5)

### Build a Machine Learning Model

In [ ]:
X = df12.drop(['price'], axis='columns')
y = df12.price

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
# create a regressor object
regressor = DecisionTreeRegressor(random_state = 0) 
  
# fit the regressor with X and Y data
regressor.fit(X, y)

### Test the Model

In [ ]:
def predict_price(location,baths,area_sqft,bedrooms):    
    loc_index = np.where(X.columns==location)[0][0]
    x = np.zeros(len(X.columns))
    x[0] = baths
    x[1] = area_sqft
    x[2] = bedrooms
    if loc_index >= 0:
        x[loc_index] = 1
    return regressor.predict([x])[0]

In [ ]:
#First argument is location
#Second argument is bathrooms
#Third argument is area of square feet
#Last and fourth argument is bedrooms
print("Rs:",predict_price('Bahria Town Karachi',5, 2778, 5))

<h1 style='color:purple' align='center'>Thank you so much for taking the time to look at my notebook. Please upvote if you like my Notebook.<h1/>